# Testing a hypothesis with maximum likekihood

Let's say we had some data, and we wanted to generate some statistical information about it and the potential patterns it might follow. 

In this case, we have a file called "chickwts.txt", containing a bunch of rows and columns of data, with information separated by commas. The first column of this dataset indicates the measured **weight** of a whole bunch of chicks, and the second column indicates what kind of **feed** they were on (horsebean, linseed, soybean, etc.) How could we start analyzing this data and looking for patterns?

## Import necessary packages

As usual, when working in Python, we want to make sure we have the right packages installed to make working with our data easier. Since we're working with a bunch of numbers and tables, it seems likely we'll need both numpy and pandas. As it turns out, we'll also want to look at some graphical representations, so we'll want plotnine, and we'll also end up doing some statistical calculations, so we'll want some elements of scipy.


In [ ]:
import numpy
import pandas
from plotnine import *
from scipy.optimize import minimize
from scipy.stats import norm

## Import dataset

Conveniently, the dataset we're working with is delineated by commas and has a header line labeling the columns, making it pretty easy to import into Python. We'll import it as a pandas dataframe and call it something convenient, like "weights""

In [ ]:
weights=pandas.read_csv("chickwts.txt", header=0, sep=",")

## Create a plot to summarize the data

Ideally, we'd like to plot each individual **feed** type separately, so we could see if there was a pattern or connection between how much the chicks **weigh** and what type of **feed** they're getting. To do this, we can separate the larger dataframe into a series of smaller dataframes, each representing one type of **feed**:

In [ ]:
horsebean=weights.loc[weights.feed.isin(['horsebean']),:]
linseed=weights.loc[weights.feed.isin(['linseed']),:]
soybean=weights.loc[weights.feed.isin(['soybean']),:]
sunflower=weights.loc[weights.feed.isin(['sunflower']),:]
meatmeal=weights.loc[weights.feed.isin(['meatmeal']),:]
casein=weights.loc[weights.feed.isin(['casein']),:]

Then, we'll create an initial plotnine object with one of the individual datasets - say, the horsebean dataframe. We'll graph the data on a scatter plot, with individual data points as separate points on the graph, to better see variation between individuals within the same group. We'll also label the x and y axes, and tell Python we'd like to just keep it simple:

In [ ]:
g=ggplot(horsebean,aes(x='feed',y='weight'))+geom_point()+theme_classic()

Once this initial plotnine object exists, we can add more data to it, simply by adding plots for each new dataset onto the same object:

In [ ]:
g=g+geom_point(linseed, aes(x='feed',y='weight'))+theme_classic()
g=g+geom_point(soybean, aes(x='feed',y='weight'))+theme_classic()
g=g+geom_point(sunflower, aes(x='feed',y='weight'))+theme_classic()
g=g+geom_point(meatmeal, aes(x='feed',y='weight'))+theme_classic()
g=g+geom_point(casein, aes(x='feed',y='weight'))+theme_classic()

Our final step will be to actually give Python the command to display the finished plot:

In [ ]:
print(g)

## Make some hypotheses

Looking at this data, we can start to make some hypotheses about the effect of **feed** type on chick **weight**. For instance, we might want to know if there's an overall difference in chick weight when the chicks are given soybean feed versus sunflower feed.

Put another way, we could define two hypotheses for feeding chicks soybean versus sunflower feed:

*Null = There is no difference in chick weight whether they are fed with soybean or sunflower feed; the **feed** type has no effect on **weight**.*

*Alternative = There is a difference in chick weight when they are fed with soybean versus sunflower feed; the **feed** type has some effect on **weight**.*

We've learned a pretty good way to test a null versus alternative hypothesis - by estimating the maximum likelihood of each model.

## Rearrange some data

In order to directly compare these two datasets together, we have to combine them. We'll first place the two individual **feed**-specific dataframes we created above into a list, then tell pandas we'd like to concatenate the two dataframes (essentially, just add one to the end of the other):

In [ ]:
frames=[soybean,sunflower]

df=pd.concat(frames)

Right now, one of the columns in our dataframe contains a string - the name of the **feed** type the chicks have been recieving. However, this won't work in later steps - we'll need to plug this dataframe into an equation to determine if the **feed** type has an effect on **weight**. In order to do this, we'll need to substitute the names of the **feed** type with numbers.

In this case, because there's only two different types of **feed**, this is a binary system - in other words, the chicks are either recieving one type of food, or they're recieving the other. This is actually pretty easy to translate into numbers - we'll simply assign one of the feed types to a value of "1" and the other to a value of "0" (it doesn't actually matter which is which, just that they're different).

In [ ]:
df.replace(['soybean','sunflower'],[0,1],inplace=True)

## Define custom likelihood functions

We can now define some custom likelihood functions that describe our two hypotheses. Our null hypothesis function should only have one parameter - that is, it describes the y-variable (chick **weight**) as some function of parameters, with no input from whether the chicks are being fed soybean or sunflower **feed**:

In [ ]:
def null(p,obs):
    B0=p[0]
    sigma=p[1]
    expected=B0
    nll=-1*norm(expected,sigma).logpdf(obs.weight).sum()
    return nll

In contrast, our alternative hypothesis function should have an additional parameter - one that varies depending on whether the chicks were fed soybean or sunflower **feed** (whether the column reads 0 or 1):

In [ ]:
def alt(p,obs):
    B0=p[0]
    B1=p[1]
    sigma=p[2]
    expected=B0+B1*obs.feed
    nll=-1*norm(expected,sigma).logpdf(obs.weight).sum()
    return nll

## Estimate parameters by minimizing negative log likelihood

Now that we've defined some custom functions and asked them to return the negative log likelihood, we can attempt to find the best parameters for each function that match the data we already have. We do this by passing the functions some initial values (it doesn't really matter what these are, they're just a starting point for the functions to start their estimates) and asking it to iterate upon these numbers, changing them a little bit each time, until they find the values that yield the minimum value of log likelihood.

In [ ]:
initialVals1=numpy.array([1,1,1])

fitNull=minimize(null,initialVals1,method="Nelder-Mead",options={'disp':True},args=df)
fitAlt=minimize(alt,initialVals1,method="Nelder-Mead",options={'disp':True},args=df)

If we want to know what the ultimate parameters the functions calculated, we can ask Python to print the attribute 'x' for each function, which is a list of the calculated parameter values:

In [ ]:
print(fitNull.x)
print(fitAlt.x)

## Determine if the effect is statistically significant

Now that we have two values for likelihood, we can actually determine if the difference between two conditions is statistically significant by performing a Chi-square test to determine if two times the difference in likelihoods (D) is large relative to a chi-squared distribution with one degree of freedom.

We actually need to import one additional function from scipy that lets us do this - aptly named chi2:

In [ ]:
from scipy.stats import chi2

D=(2*(fitNull.fun-fitAlt.fun))

chi2feed=(1-chi2.cdf(x=D,df=1))

print(chi2feed)

Turns out, there is a statistically significant difference in chick **weight** that is dependent on the type of **feed** they're getting - soybean or sunflower.